## 16S Amplicon Demultiplex Workflow

* Chenghao Zhu
* 2019/9/18
* chhzhu@ucdavis.edu

This workflow covers the basic steps of processing 16S sequencing data, including demultiplex, filtering, and trimming primers for the raw fastq files. The demultiplexed data is then ready for otu picking. This workflow is designated for the old sequencing method from David [Miles lab](http://mills.ucdavis.edu/), which only use barcodea on the forward primer (that means the reverse (downstream) primer is not barcoded). 

In this workflow, the raw paired end read fastq files were first demultiplexed using the barcode to pick up reads that have barcode in the begining of R1. Then the unmatched reads (unmatched_R1.fastq, unmatched_R2.fastq) were demultiplexed using barcode as reverse barcode, to pick up reads that have barcode in the begining of R2. The command line tool [**fastq_multx**](https://github.com/brwnj/fastq-multx) is used to demultiplex the sequencing reads without merging then (one example that merge while demultiplexing is [PEAR](https://sco.h-its.org/exelixis/web/software/pear/doc.html)). The demultiplexed reads (sample01_R1.fastq, sample-1_R2.fastq, ...) were then filtered using a python script, to remove reads that don't have the primers in the right place (most likely generated because of errors). Primers were then cut off from each end by specifying the lengths of priimers, and the 2 fastq files that belong to the same sample were concatenated together. In the very last step, FastQC is used to check the quality of reads, to determine the length to use in [**DADA2**](https://benjjneb.github.io/dada2/tutorial.html).

This workflow requires around 40G disk space. The actual disk space might vary depands on the sample size. Make sure your hvae at lease **50G** of empty disk space before you start.

This workflow is writen in Jupyter notebook. If you want to run directly in shell command, please remove the "!" in front of each command. The "!" is a trick in Jupyter Notebook to exacute shell commands.

### Step 0. Set up

First set up the project, by creating a folder named "00-Raw" that stores the raw sequences fastq files as well as sample metadata files.

### Step 1. Demultiplex the forward and reversed separately

16S sequencing method mixes samples, sometimes from different studies, into a pool that they call it "library", after PCR amplication. Samples are labeled using unique 6-digit (sometimes 8-digit) barcode previous to pooling. And those barcodes are what used to match the sequencing reads to its co-responding sample ID. And this process is called *demultiplexing*.

Many popular demultiplexing tools like [PEAR](https://sco.h-its.org/exelixis/web/software/pear/doc.html) merges the forward and reversed reads together while demultiplexing. A consequence of that is that the quality scores are all removed. The popular otu clustering algorithm [DADA2](https://benjjneb.github.io/dada2/tutorial.html) however uses the quality scores to remove noises. 

So here we use the command line tool [fastq-multx](https://github.com/brwnj/fastq-multx) to demultiplx and keep the forward and reversed read separated. Read the fastq-multx documentation for more information.

<div class="alert alert-block alert-warning">
<b>Warning:</b> This command generates around 17G fastq files for 40 samples. Make sure you enough disk space..
</div>

In [ ]:
BARCODE_FILE="00-Raw/2017_AZ_barcodes_egg.txt"
FASTQ_LIB_R1="00-Raw/FFUBS-Run_S1_L001_R1_001.fastq"
FASTQ_LIB_R2="00-Raw/FFUBS-Run_S1_L001_R2_001.fastq"

In [ ]:
mkdir -p "01-Demultx"
mkdir -p "01-Demultx/R1"
mkdir -p "01-Demultx/R2"

#### 1.1 Demultiplex the forward

In [ ]:
fastq-multx -m 0 -x -b -B $BARCODE_FILE \
        $FASTQ_LIB_R1 $FASTQ_LIB_R2 \
        -o 01-Demultx/R1/%_R1.fastq \
        -o 01-Demultx/R1/%_R2.fastq

#### 1.2 Demultiplex the reversed

<div class="alert alert-block alert-warning">
<b>Warning:</b> This command geneerates additional 21G fastq files for 40 samples.
</div>

In [ ]:
fastq-multx -B $BARCODE_FILE -m 0 -x -b\
        01-Demultx/R1/unmatched_R2.fastq \
        01-Demultx/R1/unmatched_R1.fastq \
        -o 01-Demultx/R2/%_R2.fastq \
        -o 01-Demultx/R2/%_R1.fastq

<div class="alert alert-block alert-success">
<b>Tip:</b> The raw fastq fiels can be deleted now to save your disk space. As well as the unmatched_R1.fastq and unmatched_R2.fastq.
</div>

### Step 2. Filter

Although the demultiplex step above picks up sequences that only starts with the barcodes for each sample. However, some **PhiX** sequences that happen to start with nucleotides that are same as the barcode can also be picked up. The purpose of this step is filter out those reads, and only keep the reads that not only have barcodes, but also have both forward and reverse primer at the correct location of the sequences.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Make sure the remove_phix.py file is at the correct place.
</div>

In [ ]:
mkdir -p 02-Filter
mkdir -p 02-Filter/R1
mkdir -p 02-Filter/R2

In [ ]:
ls 01-Demultx/R1/EG*_R1.fastq | cut -f3 -d '/' |cut -f1 -d '.' >filt_R1.txt
ls 01-Demultx/R1/EG*_R2.fastq | cut -f3 -d '/' |cut -f1 -d '.' >filt_R2.txt

In [ ]:
python phix_filter.py \
    --input-forward-list filt_R1.txt \
    --input-reverse-list filt_R2.txt \
    --input-path 01-Demultx/R1 \
    --output-path 02-Filter/R1 \
    --barcodes $BARCODE_FILE \
    --forward-primer GTGTGCCAGCMGCCGCGGTAA \
    --reverse-primer GGACTACNVGGGTWTCTAAT

In [ ]:
python phix_filter.py \
    --input-forward-list filt_R2.txt \
    --input-reverse-list filt_R1.txt \
    --input-path 01-Demultx/R2 \
    --output-path 02-Filter/R2 \
    --barcodes $BARCODE_FILE \
    --forward-primer GTGTGCCAGCMGCCGCGGTAA \
    --reverse-primer GGACTACNVGGGTWTCTAAT

### Step 3. Trim off primers

In [ ]:
ls 02-Filter/R1/EG*_R1.filt.fastq | cut -f3 -d '/' | cut -f1 -d '.' > trim_R1.txt
ls 02-Filter/R1/EG*_R2.filt.fastq | cut -f3 -d '/' | cut -f1 -d '.' > trim_R2.txt

In [ ]:
mkdir -p 03-Trim
mkdir -p 03-Trim/R1
mkdir -p 03-Trim/R2

In [ ]:
while read trim
do
    fastx_trimmer -f 30 -i 02-Filter/R1/$trim.filt.fastq -o 03-Trim/R1/$trim.trim.fastq
done < trim_R1.txt

In [ ]:
while read trim
do
    fastx_trimmer -f 21 -i 02-Filter/R1/$trim.filt.fastq -o 03-Trim/R1/$trim.trim.fastq
done < trim_R2.txt

In [ ]:
while read trim
do
    fastx_trimmer -f 21 -i 02-Filter/R2/$trim.filt.fastq -o 03-Trim/R2/$trim.trim.fastq
done < trim_R1.txt

In [ ]:
while read trim
do
    fastx_trimmer -f 30 -i 02-Filter/R2/$trim.filt.fastq -o 03-Trim/R2/$trim.trim.fastq
done < trim_R2.txt

### Step 4. Concatenate

This step is only 

In [ ]:
mkdir -p 04-Combine

In [ ]:
ls 03-Trim/R1/EG*.fastq | cut -f3 -d '/' |cut -f1 -d '_' | uniq > sample_list.txt

In [ ]:
wc -l sample_list.txt

In [ ]:
while read id
do
    cat 03-Trim/R1/${id}_R1.trim.fastq 03-Trim/R2/${id}_R1.trim.fastq > 04-Combine/${id}_R1_combine.fastq
done < sample_list.txt

In [ ]:
while read id
do
    cat 03-Trim/R1/${id}_R2.trim.fastq 03-Trim/R2/${id}_R2.trim.fastq > 04-Combine/${id}_R2_combine.fastq
done < sample_list.txt

### Step 5. FastQC

In [ ]:
mkdir -p 05-FastQC

In [ ]:
fastqc --help

In [ ]:
cat 04-Combine/EG*_R1_combine.fastq | gzip -c > 05-FastQC/R1_all.fastq.gz
cat 04-Combine/EG*_R2_combine.fastq | gzip -c > 05-FastQC/R2_all.fastq.gz

In [ ]:
fastqc 05-FastQC/R1_all.fastq.gz 05-FastQC/R2_all.fastq.gz -o 05-FastQC

In [ ]:
rm 05-FastQC/R1_all.fastq.gz
rm 05-FastQC/R2_all.fastq.gz

In [ ]:
ls 05-FastQC

### Next..

Up to now, the demultiplexing is done. The sequencing library is demultipled according to the barcodes provided, into each individual sample. The folder 04-Combine has all the individual fastq files, and can be then used for OTU/ASV clustering. Although there are several options out there, a [DADA2](https://benjjneb.github.io/dada2/) workflow is provided and can be run in R.